In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
# make sure we can see/access data
!ls ~/CORD19v3

cord_19_embeddings	 document_parses  metadata.csv
COVID.DATA.LIC.AGMT.pdf  json_schema.txt  metadata.readme


In [6]:
root_path = '/home/eugene_chuvyrov/CORD19v3/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [7]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157712 entries, 0 to 157711
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          157712 non-null  object 
 1   sha               71177 non-null   object 
 2   source_x          157712 non-null  object 
 3   title             157665 non-null  object 
 4   doi               119919 non-null  object 
 5   pmcid             74956 non-null   object 
 6   pubmed_id         106010 non-null  object 
 7   license           157712 non-null  object 
 8   abstract          116229 non-null  object 
 9   publish_time      157697 non-null  object 
 10  authors           151994 non-null  object 
 11  journal           149012 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  31151 non-null   object 
 14  arxiv_id          1952 non-null    object 
 15  pdf_json_files    71177 non-null   object 
 16  pmc_json_files    54

In [8]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

129922

In [9]:
# define a class to hold info about each paper
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            self.references = []
            
            # Abstract
            if 'abstract' in content:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            if 'Abstract' in  content:
                for entry in content['Abstract']:
                    self.abstract.append(entry['text'])
            
            # Body text
            for entry in content['body_text']:
                self.body_text.append('--BEGIN PARAGRAPH---')
                self.body_text.append(entry['text'])
                self.body_text.append('--END PARAGRAPH---')
            #References
            for key, reference in content['bib_entries'].items():
                self.references.append(reference['title'])
                
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
    
first_row = FileReader(all_json[0])
print(first_row.body_text)

--BEGIN PARAGRAPH---
Antibiotics target essential bacterial physiology and biochemistry, causing microbial cell death or the cessation of growth. There are five major antibiotic targets: the bacterial cell wall, the cell membrane, protein synthesis, DNA and RNA synthesis, and folic acid (vitamin B9) metabolism ( Figure 1 ). These bacterial targets are different or nonexistent in eukaryotic cells (including those of humans), which means that antibiotics are relatively nontoxic drugs. For example, the β-lactam antibiotics such as penicillins, cephalosporins and carbapenems block the synthesis of the bacterial cell wall. This structure is absent in higher organisms but is essential for bacterial survival. The bacterial ribosome is the target of the tetracycline, aminoglycoside, macrolide and other antibiotics, and is sufficiently different from the eukaryotic ribosome that cross-inhibition does not occur.
--END PARAGRAPH---
--BEGIN PARAGRAPH---
Resistance to antibiotics occurs through fou

In [8]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [9]:
# loop through all papers and populate FileReader objects
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'references': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)

    # print('processing file ' + entry)
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['references'].append(content.references)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information
    title = meta_data['title'].values[0]
    dict_['title'].append(title)
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'references', 'abstract_summary'])
df_covid.head()

Processing index: 0 of 103314
Processing index: 10331 of 103314
Processing index: 20662 of 103314
Processing index: 30993 of 103314
Processing index: 41324 of 103314
Processing index: 51655 of 103314
Processing index: 61986 of 103314
Processing index: 72317 of 103314
Processing index: 82648 of 103314
Processing index: 92979 of 103314
Processing index: 103310 of 103314


,paper_id,abstract,body_text,authors,title,journal,references,abstract_summary
0,1c363c20418dda4f6c674f6d7af16f92b5dce974,,--BEGIN PARAGRAPH---\nAntibiotics target essen...,"Wright, Gerard D",Q&A: Antibiotic resistance: where does it come...,BMC Biol,[Antibiotic-resistant bugs in the 21st century...,Not provided.
1,c9064b5f43ac8a821197ff5a7240b5705939a14e,,--BEGIN PARAGRAPH---\nThere are good reasons t...,"Domingo, Esteban",Chapter 6 Virus Population Dynamics Examined w...,Virus as Populations,[Mutational and fitness landscapes of an RNA v...,Not provided.
2,1d7bf204420a6ba3099dd213d8f65daa550b4cd5,,--BEGIN PARAGRAPH---\nA novel respiratory path...,"WIN, M. K.. CHOW, A....",Risk assessment and laboratory investigation o...,Epidemiol Infect,[The Middle East respiratory syndrome coronavi...,Not provided.
3,9301787667eb36e650623c8dc753399f3e1b4b7c,Forty-two cytopathic effect (CPE)-positive iso...,--BEGIN PARAGRAPH---\nNext-generation sequenci...,"Gong, Yu-Nong. Chen, Guang-Wu...",A Next-Generation Sequencing Data Analysis Pip...,PLoS One,[A novel rhabdovirus associated with acute hem...,Forty-two cytopathic effect (CPE)-positive<br...
4,17a2708a7064981d2744b25a1f7794e789b5726c,,--BEGIN PARAGRAPH---\nare coronary blood flow ...,"Bauer, Maria",Cardiovascular Anatomy and Pharmacology,Basic Sciences in Anesthesia,"[Inhibits Na efflux during phase 0, Shortens d...",Not provided.


In [10]:
df_covid['abstract_word_count'] = df_covid['abstract'].apply(lambda x: len(x.strip().split()))
df_covid['body_word_count'] = df_covid['body_text'].apply(lambda x: len(x.strip().split()))
df_covid.head()

,paper_id,abstract,body_text,authors,title,journal,references,abstract_summary,abstract_word_count,body_word_count
0,1c363c20418dda4f6c674f6d7af16f92b5dce974,,--BEGIN PARAGRAPH---\nAntibiotics target essen...,"Wright, Gerard D",Q&A: Antibiotic resistance: where does it come...,BMC Biol,[Antibiotic-resistant bugs in the 21st century...,Not provided.,0,3239
1,c9064b5f43ac8a821197ff5a7240b5705939a14e,,--BEGIN PARAGRAPH---\nThere are good reasons t...,"Domingo, Esteban",Chapter 6 Virus Population Dynamics Examined w...,Virus as Populations,[Mutational and fitness landscapes of an RNA v...,Not provided.,0,11636
2,1d7bf204420a6ba3099dd213d8f65daa550b4cd5,,--BEGIN PARAGRAPH---\nA novel respiratory path...,"WIN, M. K.. CHOW, A....",Risk assessment and laboratory investigation o...,Epidemiol Infect,[The Middle East respiratory syndrome coronavi...,Not provided.,0,1753
3,9301787667eb36e650623c8dc753399f3e1b4b7c,Forty-two cytopathic effect (CPE)-positive iso...,--BEGIN PARAGRAPH---\nNext-generation sequenci...,"Gong, Yu-Nong. Chen, Guang-Wu...",A Next-Generation Sequencing Data Analysis Pip...,PLoS One,[A novel rhabdovirus associated with acute hem...,Forty-two cytopathic effect (CPE)-positive<br...,270,5231
4,17a2708a7064981d2744b25a1f7794e789b5726c,,--BEGIN PARAGRAPH---\nare coronary blood flow ...,"Bauer, Maria",Cardiovascular Anatomy and Pharmacology,Basic Sciences in Anesthesia,"[Inhibits Na efflux during phase 0, Shortens d...",Not provided.,0,22951


In [11]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52471 entries, 0 to 52470
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   paper_id             52471 non-null  object
 1   abstract             52471 non-null  object
 2   body_text            52471 non-null  object
 3   authors              51301 non-null  object
 4   title                52471 non-null  object
 5   journal              48182 non-null  object
 6   references           52471 non-null  object
 7   abstract_summary     52471 non-null  object
 8   abstract_word_count  52471 non-null  int64 
 9   body_word_count      52471 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 4.0+ MB


In [12]:
df_covid['abstract'].describe(include='all')

count     52471
unique    37150
top            
freq      15128
Name: abstract, dtype: object

In [13]:
# remove duplicates (based on identical abstract)
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

count     52339
unique    37150
top            
freq      15034
Name: abstract, dtype: object

In [14]:
df_covid.head()

,paper_id,abstract,body_text,authors,title,journal,references,abstract_summary,abstract_word_count,body_word_count
0,1c363c20418dda4f6c674f6d7af16f92b5dce974,,--BEGIN PARAGRAPH---\nAntibiotics target essen...,"Wright, Gerard D",Q&A: Antibiotic resistance: where does it come...,BMC Biol,[Antibiotic-resistant bugs in the 21st century...,Not provided.,0,3239
1,c9064b5f43ac8a821197ff5a7240b5705939a14e,,--BEGIN PARAGRAPH---\nThere are good reasons t...,"Domingo, Esteban",Chapter 6 Virus Population Dynamics Examined w...,Virus as Populations,[Mutational and fitness landscapes of an RNA v...,Not provided.,0,11636
2,1d7bf204420a6ba3099dd213d8f65daa550b4cd5,,--BEGIN PARAGRAPH---\nA novel respiratory path...,"WIN, M. K.. CHOW, A....",Risk assessment and laboratory investigation o...,Epidemiol Infect,[The Middle East respiratory syndrome coronavi...,Not provided.,0,1753
3,9301787667eb36e650623c8dc753399f3e1b4b7c,Forty-two cytopathic effect (CPE)-positive iso...,--BEGIN PARAGRAPH---\nNext-generation sequenci...,"Gong, Yu-Nong. Chen, Guang-Wu...",A Next-Generation Sequencing Data Analysis Pip...,PLoS One,[A novel rhabdovirus associated with acute hem...,Forty-two cytopathic effect (CPE)-positive<br...,270,5231
4,17a2708a7064981d2744b25a1f7794e789b5726c,,--BEGIN PARAGRAPH---\nare coronary blood flow ...,"Bauer, Maria",Cardiovascular Anatomy and Pharmacology,Basic Sciences in Anesthesia,"[Inhibits Na efflux during phase 0, Shortens d...",Not provided.,0,22951


In [15]:
df_covid.describe()

,abstract_word_count,body_word_count
count,52339.000000,52339.000000
mean,157.998223,4916.887063
std,186.518579,10242.775015
min,0.000000,5.000000
25%,0.000000,2086.000000
50%,151.000000,3474.000000
75%,234.000000,5282.000000
max,7415.000000,289235.000000


In [16]:
df_covid.dropna(inplace=True)
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46948 entries, 0 to 52470
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   paper_id             46948 non-null  object
 1   abstract             46948 non-null  object
 2   body_text            46948 non-null  object
 3   authors              46948 non-null  object
 4   title                46948 non-null  object
 5   journal              46948 non-null  object
 6   references           46948 non-null  object
 7   abstract_summary     46948 non-null  object
 8   abstract_word_count  46948 non-null  int64 
 9   body_word_count      46948 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 3.9+ MB


In [17]:
# take butcher's knife and kill all special characters from our docs
#  this is brutal and leads to a massive information loss in our scientific corpus of data
#  but this will do for initial modeling
import re

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [18]:
# everything will be lowered (lowered-case that is)
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: lower_case(x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: lower_case(x))

In [19]:
# since classification of data for training was done manually (i.e., by reading each paragraph and selecting an appropriate category)
#  this is a feeble attempt at prioritizing which articles to read first
#  here, the ones I'll read first will be the ones that are most frequently referenced by other papers in the corpus
#  ala Google PageRank, much simplified
#  obvious weakness of this approach is the prioritization of older literature, which may contain outdated facts
def create_article_refs_counts(df_covid19):
    dict_all_refs = {}
    
    # scan through all the articles in the dataframe
    #  look at their references
    #  add +1 if the reference matches the one we've already seen before
    for idx in df_covid19.index:
        titles = df_covid19['references'][idx]
        
        for title in titles:
            if dict_all_refs.get(title) != None:
                dict_all_refs[title] = dict_all_refs[title] + 1
            else:
                dict_all_refs[title] = 1
                
    return dict_all_refs

In [34]:
from datetime import datetime, timedelta
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be parsed as a date in our expected format
    """
    
    if not isinstance(string, str):
        return False
    
    try: 
        meta_published_date = datetime.strptime(string, '%Y-%m-%d')
        return True

    except ValueError:
        return False
    
def get_top100_referenced_articles_metadata():
    dict_top100_ = {'article_title': [], 'meta_link': [], 'frequency': []}
    # now find most frequently referenced articles
    article_references = create_article_refs_counts(df_covid)
    sorted_article_refs = sorted(article_references.items(), key=lambda x: x[1], reverse=True)
        
    for index, row in meta_df.iterrows():
        meta_title = row['title']
        meta_link = row['doi']

        # some unexpected data types handling
        if isinstance(meta_title, str):
            meta_title = ''.join(meta_title.split()).lower()

        # more unexpected data types handling
        if isinstance(meta_link, str):
            if not meta_link.startswith('http://doi.org/'):
                meta_link = 'http://doi.org/' + meta_link

        for article_index in range(30000):
            # make sure the referenced article exists in our metadata
            frequency = sorted_article_refs[article_index][1]
            article_title = sorted_article_refs[article_index][0]
            frequent_title = ''.join(article_title.split()).lower()

            if(frequent_title == meta_title):
                dict_top100_['article_title'].append(article_title)
                dict_top100_['meta_link'].append(meta_link)
                dict_top100_['frequency'].append(frequency)
                
    return dict_top100_

In [35]:
top100md = get_top100_referenced_articles_metadata()
df_top100 = pd.DataFrame(top100md, columns=['article_title', 'meta_link', 'frequency'])

df_top100 = df_top100.sort_values(by=['frequency'], ascending=False)

In [36]:
top100md.keys()

dict_keys(['article_title', 'meta_link', 'frequency'])

In [37]:
df_top100.head()

,article_title,meta_link,frequency
144,Clinical course and risk factors for mortality...,http://doi.org/10.1016/s0140-6736(20)30566-3,397
81,SARS-CoV-2 Cell Entry Depends on ACE2 and TMPR...,http://doi.org/10.1016/j.cell.2020.02.052,135
69,COVID-19: consider cytokine storm syndromes an...,http://doi.org/10.1016/s0140-6736(20)30628-0,117
53,Are patients with hypertension and diabetes me...,http://doi.org/10.1016/s2213-2600(20)30116-8,106
82,SARS-CoV-2 cell entry depends on ACE2 and TMPR...,http://doi.org/10.1016/j.cell.2020.02.052,104


In [38]:
# now produce a data frame listing the articles, one paragraph per row
#  the unit of classification will be a paragraph of text, not an article
#  something that may generalize well (or not)
def get_training_set():
    dict_trainingset_ = {'paper_id': [], 'article_title': [], 'meta_link': [], 'abstract': [], 'paragraph_text': [], 'frequency': [], 'authors': [], 'references': [], 'journal': [], 'abstract_summary': []}

    for index, row in df_covid.iterrows():
        covid_title = row['title']
        
        # some unexpected data types handling
        if isinstance(covid_title, str):
            covid_title = ''.join(covid_title.split()).lower()

        article_title = row['article_title']
        meta_link = row['meta_link']
        frequency = row['frequency']
        lower_title = ''.join(article_title.split()).lower()

        article_text = row['body_text']
        paragraphs = article_text.split('begin paragraph')

        for paragraph in paragraphs:                    
            dict_trainingset_['article_title'].append(article_title)
            dict_trainingset_['meta_link'].append(meta_link)
            dict_trainingset_['frequency'].append(frequency)

            dict_trainingset_['paper_id'].append(row['paper_id'])
            dict_trainingset_['abstract'].append(row['abstract'])
            dict_trainingset_['paragraph_text'].append(paragraph.strip())
            dict_trainingset_['authors'].append(row['authors'])
            dict_trainingset_['references'].append(row['references'])
            dict_trainingset_['journal'].append(row['journal'])
            dict_trainingset_['abstract_summary'].append(row['abstract_summary'])

    return dict_trainingset_

In [39]:
training_set = get_training_set()

In [40]:
#print(training_set['article_title'])

['A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic epitope in the receptor-binding domains of SARS-CoV-2 and SARS-CoV', 'A highly conserved cryptic e

In [41]:
training_set.keys()
print(len(training_set['abstract']), len(training_set['paragraph_text']), len(training_set['frequency']), len(training_set['authors']), len(training_set['references']), len(training_set['journal']), len(training_set['abstract_summary']))


5818 5818 5818 5818 5818 5818 5818


In [42]:
final_set = pd.DataFrame(training_set, columns=['paper_id', 'article_title', 'meta_link', 'abstract', 'paragraph_text', 'frequency', 'authors', 'references', 'journal', 'abstract_summary'])
final_set.head()


,paper_id,article_title,meta_link,abstract,paragraph_text,frequency,authors,references,journal,abstract_summary
0,164ecabfeed6d7d4a2ba99ac0ca48565636fec4f,A highly conserved cryptic epitope in the rece...,http://doi.org/10.1126/science.abb7269,the outbreak of covid19 which is caused by sar...,,19,"Yuan, Meng. Wu, Nicholas C....",[Potent neutralization of severe acute respira...,bioRxiv,"The outbreak of COVID-19, which is caused by<..."
1,164ecabfeed6d7d4a2ba99ac0ca48565636fec4f,A highly conserved cryptic epitope in the rece...,http://doi.org/10.1126/science.abb7269,the outbreak of covid19 which is caused by sar...,3 such a high degree of sequence similarity ra...,19,"Yuan, Meng. Wu, Nicholas C....",[Potent neutralization of severe acute respira...,bioRxiv,"The outbreak of COVID-19, which is caused by<..."
2,164ecabfeed6d7d4a2ba99ac0ca48565636fec4f,A highly conserved cryptic epitope in the rece...,http://doi.org/10.1126/science.abb7269,the outbreak of covid19 which is caused by sar...,can bind to the rbd of sarscov2 5 this finding...,19,"Yuan, Meng. Wu, Nicholas C....",[Potent neutralization of severe acute respira...,bioRxiv,"The outbreak of COVID-19, which is caused by<..."
3,164ecabfeed6d7d4a2ba99ac0ca48565636fec4f,A highly conserved cryptic epitope in the rece...,http://doi.org/10.1126/science.abb7269,the outbreak of covid19 which is caused by sar...,implying their likely importance in the affini...,19,"Yuan, Meng. Wu, Nicholas C....",[Potent neutralization of severe acute respira...,bioRxiv,"The outbreak of COVID-19, which is caused by<..."
4,164ecabfeed6d7d4a2ba99ac0ca48565636fec4f,A highly conserved cryptic epitope in the rece...,http://doi.org/10.1126/science.abb7269,the outbreak of covid19 which is caused by sar...,out of 28 residues in the epitope defined as r...,19,"Yuan, Meng. Wu, Nicholas C....",[Potent neutralization of severe acute respira...,bioRxiv,"The outbreak of COVID-19, which is caused by<..."


In [44]:
final_set.to_csv('~/CORD19v3/test_v3.csv')